In [2]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [26]:
from sklearn.ensemble import (
    BaggingClassifier,
    GradientBoostingClassifier,
    RandomForestClassifier,
)

In [29]:
import optuna
from sklearn.model_selection import cross_val_score

In [3]:
pd.set_option("display.max_columns", 500)

In [4]:
folder_path = "../data/"

In [5]:
top_features = pd.read_csv(folder_path + "top_features.csv")

In [6]:
top_features = sorted(top_features.feature)

In [7]:
top_features

['C1',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'C2',
 'C5',
 'C6',
 'C8',
 'C9',
 'D1',
 'D10',
 'D11',
 'D12',
 'D13',
 'D14',
 'D15',
 'D2',
 'D3',
 'D4',
 'D5',
 'D6',
 'D8',
 'D9',
 'DeviceInfo',
 'M4',
 'M5',
 'M6',
 'P_emaildomain',
 'P_emaildomain_2',
 'ProductCD',
 'R_emaildomain',
 'TransactionAmt',
 'TransactionAmt_to_mean_card1',
 'TransactionAmt_to_mean_card2',
 'TransactionAmt_to_mean_card3',
 'TransactionAmt_to_mean_card4',
 'TransactionAmt_to_mean_card5',
 'TransactionAmt_to_mean_card6',
 'TransactionAmt_to_std_card1',
 'TransactionAmt_to_std_card2',
 'TransactionAmt_to_std_card3',
 'TransactionAmt_to_std_card4',
 'TransactionAmt_to_std_card5',
 'TransactionAmt_to_std_card6',
 'V126',
 'V127',
 'V128',
 'V13',
 'V130',
 'V131',
 'V20',
 'V282',
 'V283',
 'V285',
 'V291',
 'V294',
 'V306',
 'V307',
 'V308',
 'V310',
 'V312',
 'V313',
 'V314',
 'V315',
 'V317',
 'V35',
 'V38',
 'V44',
 'V45',
 'V53',
 'V54',
 'V56',
 'V61',
 'V62',
 'V75',
 'V76',
 'V78',
 'V82',
 'V83',


In [8]:
# X_test = pd.read_csv(folder_path + "X_test.csv")
# X_test

In [9]:
# X = pd.read_csv(folder_path + "X.csv")
# X

In [10]:
# target_train = pd.read_csv(folder_path + "target_train.csv")
# target_train = target_train.isFraud

In [11]:
categorical = [
    # "DeviceInfo", too much for ohe
    "DeviceType",
    "M1",
    "M2",
    "M3",
    "M4",
    "M5",
    "M6",
    "M7",
    "M8",
    "M9",
    "P_emaildomain",
    "P_emaildomain_1",
    "P_emaildomain_2",
    "P_emaildomain_3",
    "ProductCD",
    "R_emaildomain",
    "R_emaildomain_1",
    "R_emaildomain_2",
    "R_emaildomain_3",
    "card4",
    "card6",
    "id_12",
    # "id_13",
    # "id_14",
    "id_15",
    "id_16",
    # "id_17",
    # "id_19",
    # "id_20",
    "id_28",
    "id_29",
    # "id_30",
    # "id_31",
    "id_32",
    # "id_33",
    "id_34",
    "id_35",
    "id_36",
    "id_37",
    "id_38",
]

In [12]:
categorical_top_features = sorted(list(set(top_features) & set(categorical)))

In [13]:
categorical_top_features

['M4',
 'M5',
 'M6',
 'P_emaildomain',
 'P_emaildomain_2',
 'ProductCD',
 'R_emaildomain',
 'card4',
 'card6']

In [14]:
numeric_top_features = sorted(list(set(top_features) - set(categorical_top_features)))

In [15]:
len(numeric_top_features)

91

In [16]:
len(categorical_top_features)

9

NOTE: 8 columns of id will not be OHE

# upload data_train_downsample_top_features

In [17]:
data_train_downsample_top_features = pd.read_csv(
    folder_path + "data_train_downsample_top_features.csv"
)

In [18]:
data_train_downsample_top_features

,C1,C10,C11,C12,C13,C14,C2,C5,C6,C8,C9,D1,D10,D11,D12,D13,D14,D15,D2,D3,D4,D5,D6,D8,D9,DeviceInfo,TransactionAmt,TransactionAmt_to_mean_card1,TransactionAmt_to_mean_card2,TransactionAmt_to_mean_card3,TransactionAmt_to_mean_card4,TransactionAmt_to_mean_card5,TransactionAmt_to_mean_card6,TransactionAmt_to_std_card1,TransactionAmt_to_std_card2,TransactionAmt_to_std_card3,TransactionAmt_to_std_card4,TransactionAmt_to_std_card5,TransactionAmt_to_std_card6,V126,V127,V128,V13,V130,V131,V20,V282,V283,V285,V291,V294,V306,V307,V308,V310,V312,V313,V314,V315,V317,V35,V38,V44,V45,V53,V54,V56,V61,V62,V75,V76,V78,V82,V83,V87,addr1,card1,card2,card3,card5,dist1,id_01,id_02,id_05,id_06,id_13,id_19,id_20,id_30,id_31,id_33,M4,M5,M6,P_emaildomain,P_emaildomain_2,ProductCD,R_emaildomain,card4,card6,isFraud
0,1.0,0.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,14.0,13.0,13.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,2740,68.500,0.194640,0.000000,0.467667,0.257761,0.357614,0.360461,0.184566,0.000000,0.273001,0.170233,0.205289,0.202121,0.0,117.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.0,1.0,1.0,0.000000,117.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,117.0,0.542594,1.1624,1.083891,1.120779,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.00000,0.000000,1.000000,315.0,13926,0.0,150.0,142.0,19.0,0.0,0.0,0.0,0.0,55,568,547,86,136,461,2,0,1,32,7,4,32,1,1,0
1,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2740,49.000,0.365477,0.389806,0.334536,0.367973,0.347265,0.421206,0.491192,0.237929,0.195285,0.214723,0.208132,0.257389,0.0,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,272.0,5937,555.0,150.0,226.0,36.0,0.0,0.0,0.0,0.0,55,568,547,86,136,461,1,0,1,16,2,4,32,4,2,0
2,1.0,1.0,2.0,2.0,2.0,1.0,4.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,83.0,0.0,2526,75.887,1.289521,1.670997,1.412687,0.573218,0.839154,0.399333,2.636139,2.273508,1.261766,0.298868,0.374466,0.223918,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,4.000000,4.000000,2.0,4.0,0.0,166.215393,166.215393,166.215393,90.327904,90.327904,90.327904,90.327904,90.327904,0.0,0.000000,4.0000,1.000000,1.000000,0.000000,0.000000,4.000000,1.000000,1.000000,0.000000,0.000000,3.000000,1.00000,1.000000,1.000000,0.0,16496,352.0,117.0,134.0,0.0,-5.0,191631.0,0.0,0.0,42,307,42,86,46,461,0,2,2,16,2,0,16,2,1,0
3,4.0,0.0,3.0,0.0,22.0,3.0,4.0,1.0,5.0,0.0,2.0,0.0,465.0,423.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2740,57.950,0.868009,0.461005,0.395640,0.435185,0.410694,0.498140,3.931656,0.281387,0.230955,0.253943,0.246148,0.304402,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.542594,1.1624,1.083891,1.120779,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.00000,0.000000,1.000000,315.0,7055,555.0,150.0,226.0,3.0,0.0,0.0,0.0,0.0,55,568,547,86,136,461,3,2,0,32,7,4,32,4,2,0
4,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2740,39.000,0.234679,0.271014,0.266263,0.294589,0.347342,0.335245,0.165656,0.204789,0.155431,0.153595,0.198215,0.204861,0.0,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,299.0,7875,314.0,150.0,224.0,0.0,0.0,0.0,0.0,0.0,55,568,547,86,136,461,0,0,0,16,2,4,32,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [19]:
data_train_downsample_top_features = pd.read_csv(
    folder_path + "data_train_downsample_1_to_20_top_features.csv"
)

In [20]:
data_train_downsample_top_features

,C1,C10,C11,C12,C13,C14,C2,C5,C6,C8,C9,D1,D10,D11,D12,D13,D14,D15,D2,D3,D4,D5,D6,D8,D9,DeviceInfo,TransactionAmt,TransactionAmt_to_mean_card1,TransactionAmt_to_mean_card2,TransactionAmt_to_mean_card3,TransactionAmt_to_mean_card4,TransactionAmt_to_mean_card5,TransactionAmt_to_mean_card6,TransactionAmt_to_std_card1,TransactionAmt_to_std_card2,TransactionAmt_to_std_card3,TransactionAmt_to_std_card4,TransactionAmt_to_std_card5,TransactionAmt_to_std_card6,V126,V127,V128,V13,V130,V131,V20,V282,V283,V285,V291,V294,V306,V307,V308,V310,V312,V313,V314,V315,V317,V35,V38,V44,V45,V53,V54,V56,V61,V62,V75,V76,V78,V82,V83,V87,addr1,card1,card2,card3,card5,dist1,id_01,id_02,id_05,id_06,id_13,id_19,id_20,id_30,id_31,id_33,M4,M5,M6,P_emaildomain,P_emaildomain_2,ProductCD,R_emaildomain,card4,card6,isFraud
0,1.0,0.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,14.0,13.0,13.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,2740,68.500,0.194640,0.000000,0.467667,0.257761,0.357614,0.360461,0.184566,0.000000,0.273001,0.170233,0.205289,0.202121,0.0,117.0,0.0,1.000000,0.0,0.0,1.000000,1.000000,1.000000,0.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,117.0,0.542594,1.1624,1.083891,1.120779,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.00000,0.000000,1.000000,315.0,13926,0.0,150.0,142.0,19.0,0.0,0.0,0.0,0.0,55,568,547,86,136,461,2,0,1,32,7,4,32,1,1,0
1,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2740,39.000,0.234679,0.271014,0.266263,0.294589,0.347342,0.335245,0.165656,0.204789,0.155431,0.153595,0.198215,0.204861,0.0,0.0,0.0,1.000000,0.0,0.0,1.000000,1.000000,1.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,299.0,7875,314.0,150.0,224.0,0.0,0.0,0.0,0.0,0.0,55,568,547,86,136,461,0,0,0,16,2,4,32,2,2,0
2,2.0,1.0,2.0,2.0,3.0,2.0,5.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,83.0,0.0,2526,75.887,1.737993,2.039135,1.412687,0.569886,0.537815,0.399333,2.230553,2.223815,1.261766,0.332545,0.322337,0.223918,0.0,0.0,0.0,0.000000,0.0,0.0,1.000000,1.000000,1.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,5.0000,2.000000,2.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,4.000000,1.00000,1.000000,2.000000,0.0,13329,569.0,117.0,226.0,0.0,-10.0,116098.0,0.0,0.0,42,307,42,86,46,461,0,2,2,16,2,0,16,4,1,0
3,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2740,2454.000,6.018449,16.887618,16.754101,18.536461,21.855849,21.094660,2.964401,8.641732,9.780213,9.664663,12.472285,12.890484,0.0,0.0,0.0,0.000000,0.0,0.0,1.000000,1.000000,1.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,1.0000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.00000,1.000000,1.000000,184.0,2213,556.0,150.0,224.0,0.0,0.0,0.0,0.0,0.0,55,568,547,86,136,461,3,2,1,19,2,4,32,2,2,0
4,2.0,0.0,1.0,0.0,9.0,2.0,1.0,1.0,1.0,0.0,1.0,0.0,456.0,0.0,0.0,0.0,0.0,456.0,0.0,0.0,456.0,32.0,0.0,0.0,0.0,2740,60.000,0.425700,0.402966,0.409636,0.450580,0.425223,0.515762,0.298385,0.247251,0.239125,0.262927,0.254856,0.315171,0.0,0.0,0.0,0.000000,0.0,0.0,1.000000,1.000000,1.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,1.0000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.00000,1.000000,1.000000,204.0,7207,111.0,150.0,226.0,0.0,0.0,0.0,0.0,0.0,55,568,547,86,136,461,3,2,0,9,8,4,32,4,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

# try of train on all data: too long

In [56]:
from sklearn.compose import ColumnTransformer

column_transformer = ColumnTransformer(
    [
        ("ohe", OneHotEncoder(handle_unknown="ignore"), categorical_top_features),
        ("scaling", StandardScaler(), numeric_top_features),
    ]
)

X_train_transf = column_transformer.fit_transform(X)
X_test_transf = column_transformer.transform(X_test)

In [60]:
X_train_transf.shape

(590540, 246)

In [61]:
X_test_transf.shape

(506691, 246)

In [68]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X_train_transf, target_train, test_size=0.2, random_state=0, shuffle=False
)

In [71]:
# search_space = {"n_neighbors": [1, 3, 5, 8, 11, 15], "p": [i for i in range(1, 4)]}
# grid_search_knn = GridSearchCV(
#     KNeighborsClassifier(),
#     search_space,
#     scoring="roc_auc",
#     refit=True,
#     n_jobs=-1,
#     verbose=10,
# )
# grid_search_knn.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 4/5; 2/18] START n_neighbors=1, p=2.........................................
[CV 1/5; 2/18] START n_neighbors=1, p=2.........................................
[CV 1/5; 1/18] START n_neighbors=1, p=1.........................................
[CV 3/5; 1/18] START n_neighbors=1, p=1.........................................
[CV 2/5; 2/18] START n_neighbors=1, p=2.........................................
[CV 1/5; 3/18] START n_neighbors=1, p=3.........................................
[CV 5/5; 1/18] START n_neighbors=1, p=1.........................................
[CV 4/5; 1/18] START n_neighbors=1, p=1.........................................
[CV 3/5; 2/18] START n_neighbors=1, p=2.........................................
[CV 5/5; 2/18] START n_neighbors=1, p=2.........................................
[CV 2/5; 1/18] START n_neighbors=1, p=1.........................................
[CV 2/5; 3/18] START n_neighbors=1, p=3.........

KeyboardInterrupt: 

# train on downsample

In [21]:
X_downsample = data_train_downsample_top_features.drop(columns=["isFraud"])
y_downsample = data_train_downsample_top_features["isFraud"]

In [22]:
from sklearn.compose import ColumnTransformer

column_transformer = ColumnTransformer(
    [
        ("ohe", OneHotEncoder(handle_unknown="ignore"), categorical_top_features),
        ("scaling", StandardScaler(), numeric_top_features),
    ]
)

X_train_transf_downsample = column_transformer.fit_transform(X_downsample)

In [23]:
from sklearn.model_selection import train_test_split

(
    X_train_downsample,
    X_val_downsample,
    y_train_downsample,
    y_val_downsample,
) = train_test_split(
    X_train_transf_downsample,
    y_downsample,
    test_size=0.2,
    random_state=0,
    shuffle=False,
)

In [32]:
# search_space = {
#     "n_neighbors": [
#         5,
#     ],
#     "p": [i for i in range(1, 3)],
# }
# grid_search_knn = GridSearchCV(
#     KNeighborsClassifier(), search_space, scoring="roc_auc", n_jobs=-1, verbose=10, cv=4
# )
# # there so long to calc with this num of examples
# grid_search_knn.fit(X_train_downsample[::5], y_train_downsample[::5])

In [33]:
# print(f"Best parameters: {grid_search_knn.best_params_}")
# print(f"Best roc_auc score: {grid_search_knn.best_score_:.3f}")

In [40]:
def objective(trial):
    n_neighbors = trial.suggest_int("n_neighbors", 1, 10)
    p = trial.suggest_int("p", 1, 4)

    model = KNeighborsClassifier(
        n_neighbors=n_neighbors,
        p=p,
    )
    score = cross_val_score(
        model,
        X_train_downsample[::5],
        y_train_downsample[::5],
        cv=4,
        scoring="roc_auc",
        n_jobs=8,
    )
    roc_auc = score.mean()
    return roc_auc


study = optuna.create_study(
    direction="maximize",
)

study.optimize(objective, n_trials=10, show_progress_bar=True)

[I 2024-12-16 19:31:41,045] A new study created in memory with name: no-name-943a3d82-a961-4ca2-bf45-324d08de9e3a


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2024-12-16 19:31:42,620] Trial 0 finished with value: 0.7881765106198142 and parameters: {'n_neighbors': 4, 'p': 2}. Best is trial 0 with value: 0.7881765106198142.
[I 2024-12-16 19:32:16,195] Trial 1 finished with value: 0.7705856568881357 and parameters: {'n_neighbors': 8, 'p': 4}. Best is trial 0 with value: 0.7881765106198142.
[I 2024-12-16 19:32:49,015] Trial 2 finished with value: 0.7722703236385517 and parameters: {'n_neighbors': 9, 'p': 4}. Best is trial 0 with value: 0.7881765106198142.
[I 2024-12-16 19:32:49,303] Trial 3 finished with value: 0.6975072598337932 and parameters: {'n_neighbors': 1, 'p': 2}. Best is trial 0 with value: 0.7881765106198142.
[I 2024-12-16 19:32:49,692] Trial 4 finished with value: 0.7925848520636911 and parameters: {'n_neighbors': 5, 'p': 2}. Best is trial 4 with value: 0.7925848520636911.
[I 2024-12-16 19:32:53,663] Trial 5 finished with value: 0.8195505206552343 and parameters: {'n_neighbors': 7, 'p': 1}. Best is trial 5 with value: 0.8195505206

In [46]:
print(f"Лучшие параметры: {study.best_params}")
# {'n_neighbors': 7, 'p': 1}
print(f"Лучшее значение roc auc: {study.best_value}")

Лучшие параметры: {'n_neighbors': 7, 'p': 1}
Лучшее значение roc auc: 0.8195505206552343


### on val

In [ ]:
# Train on all data with this params and check at val

# Linear

In [42]:
from sklearn.svm import LinearSVC

In [43]:
search_space = {
    "loss": ["hinge", "squared_hinge"],
    "C": [0.01, 0.1, 1, 10],
}

In [44]:
# grid_search_linear = GridSearchCV(
#     LinearSVC(random_state=42, penalty="l2", max_iter=100000),
#     search_space,
#     scoring="roc_auc",
#     n_jobs=-1,
#     verbose=1,
#     cv=4,
# )
# grid_search_linear.fit(X_train_downsample, y_train_downsample)

In [ ]:
print(f"Best parameters: {grid_search_linear.best_params_}")
print(f"Best roc_auc score: {grid_search_linear.best_score_:.3f}")

In [45]:
def objective(trial):
    loss = trial.suggest_categorical("loss", ["hinge", "squared_hinge"])
    C = trial.suggest_float("C", 0.01, 5)

    model = LinearSVC(random_state=42, penalty="l2", max_iter=100000, C=C, loss=loss)
    score = cross_val_score(
        model,
        X_train_downsample,
        y_train_downsample,
        cv=4,
        scoring="roc_auc",
        n_jobs=-1,
    )
    roc_auc = score.mean()
    return roc_auc


study_linear = optuna.create_study(
    direction="maximize",
)

study_linear.optimize(objective, n_trials=10, show_progress_bar=True)

[I 2024-12-16 19:34:09,073] A new study created in memory with name: no-name-adbc29e9-0842-4430-9e8c-b1ff5d381931


  0%|          | 0/10 [00:00<?, ?it/s]

/home/mlcore/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1243: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/mlcore/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1243: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/mlcore/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1243: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/mlcore/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1243: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[I 2024-12-16 19:36:13,798] Trial 0 finished with value: 0.825058602100345 and parameters: {'loss': 'hinge', 'C': 4.408454538699902}. Best is trial 0 with value: 0.825058602100345.


/home/mlcore/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1243: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/mlcore/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1243: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/mlcore/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1243: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/mlcore/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1243: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


...

[I 2024-12-16 19:43:09,432] Trial 6 finished with value: 0.8237205556381979 and parameters: {'loss': 'hinge', 'C': 1.2200829430875377}. Best is trial 5 with value: 0.825989985015489.


/home/mlcore/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1243: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/mlcore/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1243: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/mlcore/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1243: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/mlcore/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1243: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[I 2024-12-16 19:44:12,179] Trial 7 finished with value: 0.8243133897403329 and parameters: {'loss': 'hinge', 'C': 1.950136484614543}. Best is trial 5 with value: 0.825989985015489.


/home/mlcore/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1243: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/mlcore/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1243: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/mlcore/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1243: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/mlcore/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1243: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[I 2024-12-16 19:44:52,536] Trial 8 finished with value: 0.8230626886125493 and parameters: {'loss': 'hinge', 'C': 0.6556451845625354}. Best is trial 5 with value: 0.825989985015489.
[I 2024-12-16 19:45:09,339] Trial 9 finished with value: 0.826003627347705 and parameters: {'loss': 'squared_hinge', 'C': 3.4668333753961114}. Best is trial 9 with value: 0.826003627347705.


In [47]:
print(f"Лучшие параметры: {study_linear.best_params}")
# {'n_neighbors': 7, 'p': 1}
print(f"Лучшее значение roc auc: {study_linear.best_value}")

Лучшие параметры: {'loss': 'squared_hinge', 'C': 3.4668333753961114}
Лучшее значение roc auc: 0.826003627347705


# Decision tree

In [52]:
from sklearn.tree import DecisionTreeClassifier

In [53]:
search_space = {
    "criterion": ["gini", "entropy", "log_loss"],
    "max_depth": [5, 7, 9, 12],
}

In [50]:
# grid_search_tree = GridSearchCV(
#     DecisionTreeClassifier(random_state=42),
#     search_space,
#     scoring="roc_auc",
#     n_jobs=-1,
#     verbose=10,
#     cv=4,
# )
# grid_search_tree.fit(X_train_downsample, y_train_downsample)

In [51]:
# print(f"Best parameters: {grid_search_tree.best_params_}")
# print(f"Best roc_auc score: {grid_search_tree.best_score_:.3f}")

In [55]:
def objective(trial):
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy", "log_loss"])
    max_depth = trial.suggest_int("max_depth", 5, 12)
    model = DecisionTreeClassifier(
        random_state=42, max_depth=max_depth, criterion=criterion
    )
    score = cross_val_score(
        model,
        X_train_downsample,
        y_train_downsample,
        cv=4,
        scoring="roc_auc",
        n_jobs=8,
    )
    roc_auc = score.mean()
    return roc_auc


study_tree = optuna.create_study(
    direction="maximize",
)

study_tree.optimize(objective, n_trials=20, show_progress_bar=True)

[I 2024-12-16 19:46:07,722] A new study created in memory with name: no-name-14f6247b-a927-4ecf-9373-8a3cb994a611


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2024-12-16 19:46:09,907] Trial 0 finished with value: 0.8244025776574874 and parameters: {'criterion': 'log_loss', 'max_depth': 10}. Best is trial 0 with value: 0.8244025776574874.
[I 2024-12-16 19:46:11,664] Trial 1 finished with value: 0.8311011877359784 and parameters: {'criterion': 'entropy', 'max_depth': 9}. Best is trial 1 with value: 0.8311011877359784.
[I 2024-12-16 19:46:12,633] Trial 2 finished with value: 0.8279488597953473 and parameters: {'criterion': 'log_loss', 'max_depth': 5}. Best is trial 1 with value: 0.8311011877359784.
[I 2024-12-16 19:46:14,094] Trial 3 finished with value: 0.8051324039503882 and parameters: {'criterion': 'entropy', 'max_depth': 12}. Best is trial 1 with value: 0.8311011877359784.
[I 2024-12-16 19:46:15,743] Trial 4 finished with value: 0.8051324039503882 and parameters: {'criterion': 'entropy', 'max_depth': 12}. Best is trial 1 with value: 0.8311011877359784.
[I 2024-12-16 19:46:17,172] Trial 5 finished with value: 0.8051324039503882 and param

In [56]:
print(f"Лучшие параметры: {study_tree.best_params}")
# {'n_neighbors': 7, 'p': 1}
print(f"Лучшее значение roc auc: {study_tree.best_value}")

Лучшие параметры: {'criterion': 'entropy', 'max_depth': 7}
Лучшее значение roc auc: 0.8362867413049774


# bagging
подобрать критерий информативности, минимальное число объектов в листе (число деревьев - побольше)

In [58]:
search_space = {
    "estimator__criterion": ["gini", "entropy", "log_loss"],
    "estimator__min_samples_leaf": [1, 3, 5, 7],
}

In [ ]:
# grid_search_bagging = GridSearchCV(
#     BaggingClassifier(
#         estimator=DecisionTreeClassifier(random_state=42),
#         n_estimators=200,
#         random_state=42,
#         n_jobs=-1,
#     ),
#     search_space,
#     scoring="roc_auc",
#     n_jobs=-1,
#     verbose=10,
#     cv=4,
# )
# grid_search_bagging.fit(X_train_downsample, y_train_downsample)

In [ ]:
# print(f"Best parameters: {grid_search_bagging.best_params_}")
# print(f"Best roc_auc score: {grid_search_bagging.best_score_:.3f}")

In [63]:
def objective(trial):
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy", "log_loss"])
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 7)
    model = (
        estimator=DecisionTreeClassifier(
            random_state=42, criterion=criterion, min_samples_leaf=min_samples_leaf
        ),
        n_estimators=20,
        random_state=42,
        n_jobs=4,
    )
    score = cross_val_score(
        model,
        X_train_downsample,
        y_train_downsample,
        cv=4,
        scoring="roc_auc",
        n_jobs=4,
    )
    roc_auc = score.mean()
    return roc_auc


study_bagging = optuna.create_study(
    direction="maximize",
)

study_bagging.optimize(objective, n_trials=20, show_progress_bar=True)

[I 2024-12-16 19:51:41,149] A new study created in memory with name: no-name-b8a45f86-ed47-4968-992b-56cdc510cd99


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2024-12-16 19:52:00,467] Trial 0 finished with value: 0.8889949791128808 and parameters: {'criterion': 'gini', 'min_samples_leaf': 2}. Best is trial 0 with value: 0.8889949791128808.
[I 2024-12-16 19:52:17,898] Trial 1 finished with value: 0.890695162674664 and parameters: {'criterion': 'entropy', 'min_samples_leaf': 1}. Best is trial 1 with value: 0.890695162674664.
[I 2024-12-16 19:52:34,825] Trial 2 finished with value: 0.8910600498917838 and parameters: {'criterion': 'entropy', 'min_samples_leaf': 2}. Best is trial 2 with value: 0.8910600498917838.
[I 2024-12-16 19:52:50,731] Trial 3 finished with value: 0.8909729657057547 and parameters: {'criterion': 'gini', 'min_samples_leaf': 6}. Best is trial 2 with value: 0.8910600498917838.
[I 2024-12-16 19:53:06,163] Trial 4 finished with value: 0.8913386497379181 and parameters: {'criterion': 'gini', 'min_samples_leaf': 7}. Best is trial 4 with value: 0.8913386497379181.
[I 2024-12-16 19:53:22,273] Trial 5 finished with value: 0.8936999

In [64]:
print(f"Лучшие параметры: {study_bagging.best_params}")
print(f"Лучшее значение roc auc: {study_bagging.best_value}")

Лучшие параметры: {'criterion': 'entropy', 'min_samples_leaf': 6}
Лучшее значение roc auc: 0.8939214637694358


# random forest

In [65]:
search_space = {
    "estimator__criterion": ["gini", "entropy", "log_loss"],
    "estimator__min_samples_leaf": [1, 3, 5, 7],
    "min_samples_split": [2, 4, 6],
}

In [ ]:
# grid_search_random_forest = GridSearchCV(
#     RandomForestClassifier(
#         estimator=DecisionTreeClassifier(random_state=42),
#         n_estimators=200,
#         random_state=42,
#         n_jobs=-1,
#     ),
#     search_space,
#     scoring="roc_auc",
#     n_jobs=-1,
#     verbose=10,
#     cv=4,
# )
# grid_search_random_forest.fit(X_train_downsample, y_train_downsample)

In [ ]:
# print(f"Best parameters: {grid_search_random_forest.best_params_}")
# print(f"Best roc_auc score: {grid_search_random_forest.best_score_:.3f}")

In [72]:
def objective(trial):
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
    max_features = trial.suggest_categorical("max_features", ["sqrt", "log2", 0.3])
    model = RandomForestClassifier(
        random_state=42,
        criterion=criterion,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        n_estimators=100,
        n_jobs=4,
    )

    score = cross_val_score(
        model,
        X_train_downsample,
        y_train_downsample,
        cv=4,
        scoring="roc_auc",
        n_jobs=4,
    )
    roc_auc = score.mean()
    return roc_auc


study_rf = optuna.create_study(
    direction="maximize",
)

study_rf.optimize(objective, n_trials=20, show_progress_bar=True)

[I 2024-12-16 20:10:31,720] A new study created in memory with name: no-name-d18135e6-7bba-4538-af60-46d95f3b0d6d


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2024-12-16 20:10:35,711] Trial 0 finished with value: 0.8815049248373457 and parameters: {'criterion': 'entropy', 'min_samples_leaf': 9, 'max_features': 'log2'}. Best is trial 0 with value: 0.8815049248373457.
[I 2024-12-16 20:10:57,398] Trial 1 finished with value: 0.8979296564224278 and parameters: {'criterion': 'gini', 'min_samples_leaf': 8, 'max_features': 0.3}. Best is trial 1 with value: 0.8979296564224278.
[I 2024-12-16 20:11:24,487] Trial 2 finished with value: 0.9009459811506257 and parameters: {'criterion': 'gini', 'min_samples_leaf': 1, 'max_features': 0.3}. Best is trial 2 with value: 0.9009459811506257.
[I 2024-12-16 20:11:28,491] Trial 3 finished with value: 0.8962943407486367 and parameters: {'criterion': 'entropy', 'min_samples_leaf': 2, 'max_features': 'log2'}. Best is trial 2 with value: 0.9009459811506257.
[I 2024-12-16 20:11:34,303] Trial 4 finished with value: 0.8931836814290536 and parameters: {'criterion': 'entropy', 'min_samples_leaf': 9, 'max_features': 'sqr

In [74]:
print(f"Лучшие параметры: {study_rf.best_params}")
print(f"Лучшее значение roc auc: {study_rf.best_value}")

Лучшие параметры: {'criterion': 'gini', 'min_samples_leaf': 1, 'max_features': 0.3}
Лучшее значение roc auc: 0.9009459811506257


# Grad Boosting

In [ ]:
# param_grid = {
#     "learning_rate": [0.01, 0.1, 1],
#     "max_depth": [2, 3],
#     "subsample": [0.5, 0.8, 1],
#     "max_features": ["sqrt", "log2"],
# }

# grid_search = GridSearchCV(
#     estimator=GradientBoostingClassifier(random_state=0, n_estimators=300),
#     search_space,
#     scoring="roc_auc",
#     n_jobs=-1,
#     verbose=10,
#     cv=4,
# )

# grid_search.fit(X_train, y_train)

In [76]:
def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 0.001, 1)
    subsample = trial.suggest_float("subsample", 0.85, 1)
    max_depth = trial.suggest_int("max_depth", 2, 12)
    max_features = trial.suggest_categorical("max_features", ["sqrt", "log2", 0.3])

    model = GradientBoostingClassifier(
        random_state=0,
        n_estimators=100,
        learning_rate=learning_rate,
        subsample=subsample,
        max_depth=max_depth,
        max_features=max_features,
    )

    score = cross_val_score(
        model,
        X_train_downsample,
        y_train_downsample,
        cv=4,
        scoring="roc_auc",
        n_jobs=8,
    )
    roc_auc = score.mean()
    return roc_auc


study_gb = optuna.create_study(
    direction="maximize",
)

study_gb.optimize(objective, n_trials=30, show_progress_bar=True)

[I 2024-12-16 20:23:25,849] A new study created in memory with name: no-name-4f721853-c42e-48d7-9e00-0054496ade6d


  0%|          | 0/30 [00:00<?, ?it/s]

[I 2024-12-16 20:23:28,672] Trial 0 finished with value: 0.8760974907439525 and parameters: {'learning_rate': 0.5869779514319597, 'subsample': 0.8616980421755899, 'max_depth': 3, 'max_features': 'log2'}. Best is trial 0 with value: 0.8760974907439525.
[I 2024-12-16 20:23:41,800] Trial 1 finished with value: 0.8585727180633632 and parameters: {'learning_rate': 0.8179409796551661, 'subsample': 0.9285055814046063, 'max_depth': 11, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.8760974907439525.
[I 2024-12-16 20:23:45,752] Trial 2 finished with value: 0.8495554779528041 and parameters: {'learning_rate': 0.013395137271961084, 'subsample': 0.9132905574090353, 'max_depth': 4, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.8760974907439525.
[I 2024-12-16 20:23:54,968] Trial 3 finished with value: 0.897098063599209 and parameters: {'learning_rate': 0.03439208335247732, 'subsample': 0.8870261625146236, 'max_depth': 11, 'max_features': 'log2'}. Best is trial 3 with value: 0.897098

In [77]:
print(f"Лучшие параметры: {study_gb.best_params}")
print(f"Лучшее значение roc auc: {study_gb.best_value}")

Лучшие параметры: {'learning_rate': 0.10013779492764528, 'subsample': 0.9734974900131835, 'max_depth': 9, 'max_features': 0.3}
Лучшее значение roc auc: 0.9136812893972152


In [78]:
def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 0.001, 1)
    subsample = trial.suggest_float("subsample", 0.85, 1)
    max_depth = trial.suggest_int("max_depth", 2, 12)
    max_features = trial.suggest_categorical("max_features", ["sqrt", "log2", 0.3])

    model = GradientBoostingClassifier(
        random_state=0,
        n_estimators=200,
        learning_rate=learning_rate,
        subsample=subsample,
        max_depth=max_depth,
        max_features=max_features,
    )

    score = cross_val_score(
        model,
        X_train_downsample,
        y_train_downsample,
        cv=4,
        scoring="roc_auc",
        n_jobs=8,
    )
    roc_auc = score.mean()
    return roc_auc


study_gb = optuna.create_study(
    direction="maximize",
)

study_gb.optimize(objective, n_trials=30, show_progress_bar=True)

[I 2024-12-16 20:35:33,643] A new study created in memory with name: no-name-6715746a-b1c4-4d06-be96-6cce2052de4b


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2024-12-16 20:35:44,080] Trial 0 finished with value: 0.843586807867792 and parameters: {'learning_rate': 0.9066489073674951, 'subsample': 0.8590914231977526, 'max_depth': 6, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.843586807867792.
[I 2024-12-16 20:35:52,034] Trial 1 finished with value: 0.8852800284114677 and parameters: {'learning_rate': 0.023638630885344365, 'subsample': 0.9111565914771669, 'max_depth': 7, 'max_features': 'log2'}. Best is trial 1 with value: 0.8852800284114677.
[I 2024-12-16 20:35:58,227] Trial 2 finished with value: 0.8521705672389777 and parameters: {'learning_rate': 0.016198260686895616, 'subsample': 0.9466561965478197, 'max_depth': 3, 'max_features': 'sqrt'}. Best is trial 1 with value: 0.8852800284114677.
[I 2024-12-16 20:36:01,969] Trial 3 finished with value: 0.8768112978620561 and parameters: {'learning_rate': 0.6317929491976992, 'subsample': 0.9347392194269208, 'max_depth': 3, 'max_features': 'log2'}. Best is trial 1 with value: 0.88528002

In [79]:
print(f"Лучшие параметры: {study_gb.best_params}")
print(f"Лучшее значение roc auc: {study_gb.best_value}")

Лучшие параметры: {'learning_rate': 0.13614341921546924, 'subsample': 0.9627358535570463, 'max_depth': 10, 'max_features': 0.3}
Лучшее значение roc auc: 0.9156105954421854


In [80]:
def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 0.001, 1)
    subsample = trial.suggest_float("subsample", 0.85, 1)
    max_depth = trial.suggest_int("max_depth", 2, 12)
    max_features = trial.suggest_categorical("max_features", ["sqrt", "log2", 0.3])

    model = GradientBoostingClassifier(
        random_state=0,
        n_estimators=300,
        learning_rate=learning_rate,
        subsample=subsample,
        max_depth=max_depth,
        max_features=max_features,
    )

    score = cross_val_score(
        model,
        X_train_downsample,
        y_train_downsample,
        cv=4,
        scoring="roc_auc",
        n_jobs=8,
    )
    roc_auc = score.mean()
    return roc_auc


study_gb = optuna.create_study(
    direction="maximize",
)

study_gb.optimize(objective, n_trials=20, show_progress_bar=True)

[I 2024-12-16 20:51:15,764] A new study created in memory with name: no-name-5f3a311e-2225-4106-8eca-d475d95003d8


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2024-12-16 20:51:21,524] Trial 0 finished with value: 0.8866848918519936 and parameters: {'learning_rate': 0.3734301653855022, 'subsample': 0.9346328802187807, 'max_depth': 3, 'max_features': 'log2'}. Best is trial 0 with value: 0.8866848918519936.
[I 2024-12-16 20:51:42,797] Trial 1 finished with value: 0.9039044803534889 and parameters: {'learning_rate': 0.2298366389461664, 'subsample': 0.9746625474087359, 'max_depth': 7, 'max_features': 'sqrt'}. Best is trial 1 with value: 0.9039044803534889.
[I 2024-12-16 20:53:51,768] Trial 2 finished with value: 0.9176256408487836 and parameters: {'learning_rate': 0.11916998825374048, 'subsample': 0.9304122375880862, 'max_depth': 12, 'max_features': 0.3}. Best is trial 2 with value: 0.9176256408487836.
[I 2024-12-16 20:55:06,165] Trial 3 finished with value: 0.9124566564820745 and parameters: {'learning_rate': 0.13489806886681427, 'subsample': 0.9115561837039293, 'max_depth': 7, 'max_features': 0.3}. Best is trial 2 with value: 0.9176256408487

In [81]:
print(f"Лучшие параметры: {study_gb.best_params}")
print(f"Лучшее значение roc auc: {study_gb.best_value}")

Лучшие параметры: {'learning_rate': 0.04922484544238698, 'subsample': 0.9635175126291756, 'max_depth': 12, 'max_features': 0.3}
Лучшее значение roc auc: 0.9188111178272939


# stacking

In [25]:
from sklearn.ensemble import StackingClassifier

In [27]:
base_estimators = [
    ("knn", KNeighborsClassifier(n_neighbors=7, p=1)),
    (
        "random_forest",
        RandomForestClassifier(
            random_state=42,
            criterion="gini",
            min_samples_leaf=1,
            max_features=0.3,
            n_estimators=100,
            n_jobs=4,
        ),
    ),
    (
        "gradient_boosting",
        GradientBoostingClassifier(
            random_state=0,
            n_estimators=300,
            learning_rate=0.049,
            subsample=0.963,
            max_depth=12,
            max_features=0.3,
        ),
    ),
]

stacking_clf = StackingClassifier(
    estimators=base_estimators, final_estimator=GradientBoostingClassifier(), n_jobs=8
)
# stacking_clf.fit(X_train_downsample, y_train_downsample)

In [30]:
score = cross_val_score(
    stacking_clf,
    X_train_downsample,
    y_train_downsample,
    cv=4,
    scoring="roc_auc",
    n_jobs=8,
)
roc_auc = score.mean()

In [31]:
roc_auc

0.9198285185079728

In [92]:
# print(f"roc auc of stacking: {roc_auc_score(y_val_downsample, stacking_clf.predict_proba(X_val_downsample)[:,1])}")
# roc auc of stacking: 0.9153554342760114


roc auc of stacking: 0.9153554342760114


In [ ]:
roc_auc_score(y_val, knn_on_all.predict_proba(X_val))